## Additional End of week Exercise - week 2
Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
"""
Project Description: It will be an investment advisor that does the following:
1. Providing advice on suitable investment products, focused ONLY on ETFs, tailored to the client's risk tolerance and financial goals. --> For example,
it helps the user with investment recommendations based on their questions which could be something like "What ETF is there that tells me the price of the Nasdaq." An answer to that question 
can be something like "An ETF that tracks the Nasdaq is QQQ offered by Invesco.". 
2. Portfolio Management: Monitoring and adjusting client portfolios to ensure they align with their financial goals and market conditions. --> Place orders to buy ETFs in a fake account 
   + Keep track in the form of a pie chart of the consistancy of the portfolio, what it's made of ,w ith the total fake amount invested NOTHING ELSE JUST THESE 2 functionalities it will have
3. Compliance and Ethics: Adhering to regulatory requirements and maintaining high ethical standards in their practice. 
4. Doesn't answer if it doesn't know the answer

Tools available: 
 - Get_ETF_Name(prompt): Use the alpaca API to fetch an ETF the user asked for with a graph of that ETF price over time --> Output is a list of ETF index code
 - Get_ETF_Price(ETF_Index): Use an API to fetch the price of the ETF (Live Graph + current price)
 - Place_ETF_Order(ETF_Index): Use an API to place order to buy an ETF at Market price
 - Display_Portfolio(Account_info): Use API to get the information of the portfolio consistency of a person's investment account

LLM Models:
 - OpenAI and Anthropic: Have a drop down to choose the model to use (Start with OpenAI then expand to add Anthropic as well)

Added Functionality:
 - Add audio functionality: Make it so that user can converse with Assistant (AI) with audio and his voice. <--> Make that later after finishing the main features above

"""

'\nProject Description: It will be an investment advisor that does the following:\n1. Providing advice on suitable investment products, focused ONLY on ETFs, tailored to the client\'s risk tolerance and financial goals. --> For example,\nit helps the user with investment recommendations based on their questions which could be something like "What ETF is there that tells me the price of the Nasdaq." An answer to that question \ncan be something like "An ETF that tracks the Nasdaq is QQQ offered by Invesco.". \n2. Portfolio Management: Monitoring and adjusting client portfolios to ensure they align with their financial goals and market conditions. --> Place orders to buy ETFs in a fake account \n   + Keep track in the form of a pie chart of the consistancy of the portfolio, what it\'s made of ,with the total fake amount invested\n3. Compliance and Ethics: Adhering to regulatory requirements and maintaining high ethical standards in their practice. \n4. Doesn\'t answer if it doesn\'t know

Imports

In [272]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, OrderType, TimeInForce
from alpaca.data.requests import StockLatestTradeRequest
import plotly.express as px
import plotly.io as pio

In [273]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if alpaca_api_key:
    print(f"Alpaca API Key exists and begins {alpaca_api_key[:8]}")
else:
    print("Alpaca API Key not set")

if alpaca_secret_key:
    print(f"Alpaca Secret Key exists and begins {alpaca_secret_key[:8]}")
else:
    print("Alpaca Secret Key not set")

OpenAI API Key exists and begins sk-proj-
Alpaca API Key exists and begins PKGSD3LV
Alpaca Secret Key exists and begins f8sjWfsP


Step 1: Create the functions for

1. Placing a buy order --> Done
3. Seeing Portfolio information in a Pie chart --> Done
4. Get the ETF Price
5. Get ETF Name <-- For this one need to use a combo of LLM and the API from Alpaca ???? OR JUST USE THE LLM?????

In [163]:
# If you want to use paper account instead of live account, set `paper=True`
trade_client = TradingClient(api_key=alpaca_api_key, secret_key=alpaca_secret_key, paper=True, url_override=None)

In [223]:
# Function to get remaining balance, invested amount, and in what the invested amount is invested in
def get_portfolio():
    account_info = trade_client.get_account() # Get account info which includes cash on hand
    rem_amt_to_trade = account_info.non_marginable_buying_power # Amount that can be traded (It excludes the pending orders)
    all_positions = trade_client.get_all_positions()
    amt_invested = {} # Key (Symbol of asset): Total Amount invested for that specific asset
    for position in all_positions:
        symbl = position.symbol # Ticker symbol of position
        market_value = position.market_value
        qty = position.qty_available # Quantity available to sell
        amt_invested[symbl] = qty*market_value

    print("Portfolio function was called")
    return rem_amt_to_trade, amt_invested

In [211]:
# OpenAI's a particular dictionary structure that's required to describe our function:

get_portfolio_function = {
    "name": "get_portfolio",
    "description": (
        "Retrieve the user's portfolio information, including remaining account balance "
        "(non-marginable buying power), total invested amount, and a breakdown of investments "
        "per symbol. Call this when you need to know how much the user can still invest and how "
        "their current holdings are distributed."
    ),
    "additionalProperties": False
}

In [222]:
def place_buy_order(symbol, qty, type=OrderType.MARKET, time_in_force=TimeInForce.DAY, side=OrderSide.BUY):
    req = MarketOrderRequest(symbol=symbol, qty=qty, type=type, time_in_force=time_in_force, side=side)

    res = trade_client.submit_order(req)
    print("Buy order went through")

    return res

In [213]:
place_buy_order_function={
    "name": "place_buy_order",
    "description": "Post a buy order request for a symbol and amount or quantity that is specified by the user. Call this whenever you need to place a buy order request to buy ETF securities using the amount specified by the user",
    "parameters": {
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The ticker symbol of the security the user wants to buy",
            },
            "qty": {
                "type": "number",
                "description": "The number of shares the user wants to purchase. If the user specifies an investment amount in USD instead of a quantity, convert it to a share quantity by retrieving the current market price using the get_market_val function and dividing the investment amount by the price per share of the specified security." 
            },
        },
        "required": ["symbol", "qty"],
        "additionalProperties": False
    }
}

In [224]:
def get_market_val(symbol):
    req = StockLatestTradeRequest(symbol_or_symbols=[symbol])
    trade = client.get_stock_latest_trade(req)[symbol]
    market_value = trade.price
    print("Market val function was called")
    return market_value

In [233]:
get_market_val_function={
    "name": "get_market_val",
    "description": "Get the market value of the specified security using the provided 'symbol' parameter.",
    "parameters": {
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The ticker symbol of the security the user wants to look up the current market value for.",
            },
        },
        "required": ["symbol"],
        "additionalProperties": False
    }

}

Step 2: Build the LLM

In [234]:
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [243]:
system_message = """Act as an investment advisor specializing ONLY in stocks (including ETFs and individual stocks). Your job consists of two specific functions: \
1. **Stock Advice**: Provide recommendations, explanations, and product suggestions *limited to stocks* (including ETFs) that are tailored to the user's described risk tolerance, goals, or specific questions. Only include stocks that are available for trade in the United States. Do not offer advice on or refer to any \
other investment products. For every recommendation, clearly explain your reasoning step-by-step before providing the answer. Do not fabricate information. If unsure, say you do not know.\
2. **Portfolio Management Simulation**: Allow users to simulate buying stocks in a fictional portfolio. Keep an up-to-date record of simulated holdings by tracking buy orders, \
including name, ticker, amount, and value. Present the current simulated portfolio as a table breakdown showing the composition and total fake investment. If the user asks about\
their portfolio, display this chart and the breakdown in percentages and total value. **Always include the remaining amount the user can still invest in the portfolio report.** Before each portfolio report or order, briefly explain what the data represents or why the \
action is being taken, **then** display the results.\
3. **Market Value Lookup**: When the user asks for the current market value of a stock, use the tool `"get_market_val"` to retrieve the latest price and return it.\
4. **Compliance and Ethics**: Always adhere strictly to regulatory and ethical standards:\
- Never offer personalized, real-money investment advice, only general or hypothetical information.\
- Do not make up data, product details, or results. If you do not know the answer, simply state that you do not know.\
**Never respond to questions about non-stock investment products (e.g., crypto, bonds, options, real estate, etc.). Never guess. Only provide answers you are confident in.**\
---\
**Output Format:**  \
- For stock product advice:  \
  - Start with reasoning/explanation, then provide recommendation(s).\
  - Respond in concise paragraphs.  \
- For simulated portfolio management:  \
  - For each action (buy/report), first briefly explain what is happening (reasoning), then show a summarized table of holdings and total (e.g., QQQ 50%, AAPL 50%, Total: $10,000).\
  - Always include the remaining amount the user can invest (unallocated funds).\
  - Use simple ASCII graphics if needed for visual clarity.\
  - Respond in structured sections.  \
- For stock market value lookup: \
  - State that you are retrieving the latest market value, then use the `get_market_val` tool and report the result clearly.\
- If you do not know an answer or are not qualified, respond with: 'Sorry, I do not know.'  \
**Examples:**  \
Example 1:  \
User: What stock tracks the Nasdaq?  \
Reasoning: To track the Nasdaq index, an ETF must be designed to follow its performance.  \
Conclusion: The Invesco QQQ Trust (QQQ) is a stock (ETF) that tracks the Nasdaq-100 Index.\
Example 2:  \
User: Buy $5,000 of QQQ  \
Reasoning: The user requests to allocate $5,000 into QQQ. Updating the portfolio and recalculating proportions...  \
Conclusion:  \
Portfolio Breakdown:  \
- QQQ: $5,000 (100%)\
Total invested: $5,000\
Remaining amount to invest: $0\
Example 3:  \
User: Buy $5,000 of QQQ; Buy $5,000 of AAPL  \
Reasoning: The portfolio now consists of $5,000 in QQQ and $5,000 in AAPL. Here's the updated allocation:  \
Conclusion:  \
Portfolio Breakdown:  \
- QQQ: $5,000 (50%)  \
- AAPL: $5,000 (50%)\
Total invested: $10,000\
Remaining amount to invest: $0\
Example 4:  \
User: What is the current market value of AAPL?  \
Reasoning: The user is asking for the current price of Apple Inc. stock. Retrieving the latest market value using the `get_market_val` tool...  \
Conclusion: The current market value of AAPL is $193.25 per share.\
---\
**Important:**  \
- Reasoning/explanation must always precede any recommendation or portfolio result.\
- If you don't know the answer, say so.\
- Only answer about stocks and simulated portfolio management.\
- Never provide real financial or actionable advice; this is a simulated tool.\
---\
**Reminder:**  \
Focus only on stocks (ETFs and individual companies available in the U.S. market); always reason before concluding; portfolio breakdowns must always start with a brief explanation followed by the breakdown or chart; never guess or \
answer what you don't know."""


In [244]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": get_portfolio_function}, {"type": "function", "function": place_buy_order_function}, {"type": "function", "function": get_market_val_function}]

In [245]:
# We have to write that function handle_tool_call: Allows to unpack the LLM message to see what tool it needs to call

def handle_tool_call(message):
    tool_call = message.tool_calls[0] # Unpack it to see which tool it needs to call

    # If there are more than 1 tool that it can call, there needs to be an if-else condition here to see which function to call
    if tool_call.function.name == "get_portfolio":
        arguments = json.loads(tool_call.function.arguments) # Load the arguments it needs to call, to see what parameters it needs to call the function
        rem_amt_to_trade, amt_invested = get_portfolio()
        response = { # Create a response dictionary so it can be passed to the LLM and used in its response
            "role": "tool", # Tool role to inform the LLM of the tool answer
            "content": json.dumps({"remaining amt to invest": rem_amt_to_trade,"amount invested": amt_invested}), # Put in a string, json.dumps() turns a json into a string
            "tool_call_id": tool_call.id # Add in the tool_call ID --> A way to link this response to the request to call the tool it came before it in the "message" prompt made by the 
                                        # assistant
        }
        return response
    elif tool_call.function.name == "place_buy_order":
        arguments = json.loads(tool_call.function.arguments)
        symbol = arguments["symbol"]
        qty = arguments["qty"]
        res = place_buy_order(symbol, qty)  # Assumes this returns an order confirmation
        response = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps({
                "order confirmation": str(res)
            })
        }
        return response, symbol, qty

    elif tool_call.function.name == "get_market_val":
        arguments = json.loads(tool_call.function.arguments)
        symbol = arguments["symbol"]
        market_val = get_market_val(symbol)
        response = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps({
                "market value": market_val
            })
        }

    return response, symbol
    

In [256]:
# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

#     if response.choices[0].finish_reason == "tool_calls":
#         tool_message = response.choices[0].message
#         tool_name = tool_message.tool_calls[0].function.name

#         # Unpack based on which tool was called
#         if tool_name == "place_buy_order":
#             tool_response, symbol, qty = handle_tool_call(tool_message)
#         elif tool_name == "get_market_val":
#             tool_response, symbol = handle_tool_call(tool_message)
#         elif tool_name == "get_portfolio":  # get_portfolio or any other tool that returns only one value
#             tool_response = handle_tool_call(tool_message)

#         # Append assistant’s tool call and our tool response
#         messages.append(tool_message)       # Assistant says to call a tool
#         messages.append(tool_response)      # Tool responds with output

#         # Make second LLM call with updated context
#         response = openai.chat.completions.create(model=MODEL, messages=messages)

#     return response.choices[0].message.content


In [274]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        tool_message = response.choices[0].message
        tool_name = tool_message.tool_calls[0].function.name

        # Unpack based on which tool was called
        if tool_name == "place_buy_order":
            tool_response, symbol, qty = handle_tool_call(tool_message)
        elif tool_name == "get_market_val":
            tool_response, symbol = handle_tool_call(tool_message)
        elif tool_name == "get_portfolio":
            tool_response = handle_tool_call(tool_message)

        # Append assistant’s tool call and our tool response
        messages.append(tool_message)
        messages.append(tool_response)

        # Make second LLM call with updated context
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [275]:

def create_pie():
    # Optional: ensure using the default template (inherits system/theme settings)
    pio.templates.default = pio.templates.default  # no override

    rem_amt, holdings = get_portfolio()
    labels = list(holdings.keys()) + ["Cash"]
    values = list(holdings.values()) + [rem_amt]

    fig = px.pie(
        names=labels,
        values=values,
        title="Portfolio Breakdown",
        hole=0.4,
        template=None  # use whatever the default is
    )
    fig.update_traces(textinfo="label+percent")
    return fig

In [277]:
with gr.Blocks() as app:
    with gr.Row():
        with gr.Column(scale=3):
            chat_box = gr.ChatInterface(
                fn=chat,
                title="Stock Advisor Chat",
                description="Interact and simulate portfolio actions.",
                type="messages"
            )
        with gr.Column(scale=2):
            pie = gr.Plot(value=create_pie, label="Portfolio Pie Chart")

app.launch()

Portfolio function was called
* Running on local URL:  http://127.0.0.1:7884
* To create a public link, set `share=True` in `launch()`.


Portfolio function was called
